In [44]:
import astroprov
from astropy.io import fits
from astropy.table import Table
from astropy.io import ascii
from astropy.stats import sigma_clipped_stats
from astropy.coordinates import SkyCoord

import matplotlib.pyplot as plt
import numpy as np
import os
from regions.core import PixCoord
from regions.shapes.circle import CirclePixelRegion
import subprocess
import time
import matplotlib.text as mpl_text
import math
import pandas as pd
import pandasql as ps
import time

In [2]:
imageRefernce = '/home/mj1e16/moleGazer/Photos/F/29018.jpg'
imageAlign =  '/home/mj1e16/moleGazer/Photos/F/79948.jpg'

imageRefernceFits = '/home/mj1e16/moleGazer/Photos/F/fitsImages/16bit/29018InvertedAverageCropped16.fits'
imageAlignFits =  '/home/mj1e16/moleGazer/Photos/F/fitsImages/16bit/79948InvertedAverageCropped16.fits'

## max setting [6.444444444444445, 21.842105263157894, 'gauss_5.0_9x9.conv']

refTab = Table.read('/home/mj1e16/moleGazer/Photos/F/fitsImages/simMolePos/29018InvertedAverageCropped166.44444444444_21.8421052632_gauss_5.0_9x9.conv.cat',format='ascii.sextractor')
alignTab = Table.read('/home/mj1e16/moleGazer/Photos/F/fitsImages/simMolePos/79948InvertedAverageCropped166.44444444444_21.8421052632_gauss_5.0_9x9.conv.cat',format='ascii.sextractor')

In [3]:
def makeDS9RegFile(sexTabList,fileNameBase,tabType,colour='red',radius='10'):
    for tables in range(len(sexTabList)):
        if tabType == 'dao':
            xcoords = sexTabList[tables]['xcentroid']
            ycoords = sexTabList[tables]['ycentroid']
        elif tabType == 'sexMedian':
            ra = sexTabList[tables]['RA']
            dec = sexTabList[tables]['DEC']
        elif tabType == 'sex':
            xcoords = sexTabList[tables]['X_IMAGE']
            ycoords = sexTabList[tables]['Y_IMAGE']
        elif tabType == 'iraf':
            xcoords = sexTabList[tables]['X_POS']
            ycoords = sexTabList[tables]['Y_POS']        
        else:
            print('tab type error')
            break
        bigString = 'image\n'
        for x in range(len(xcoords)):
            bigString += 'circle({},{},'.format(xcoords[x],ycoords[x])+radius+') # color={}\n'.format(colour)
        fileName = fileNameBase + '.reg'
        with open(fileName,'w') as f:
            f.write(bigString)
            
    return fileName

In [27]:
def molePositions2Tab(molePositions):
    with open(molePositions,'r') as f:
        data = f.readlines()

    ycoord = []
    xcoord = []
    starLocations = []
    for x in data:
        if 'circle(' in x:
            locator1 = '('
            locator2 = ','
            loc1 = x.index(locator1) +1
            #print(x)
            loc2 = x[loc1:].index(locator2) + loc1 +1
            loc3 = x[loc2:].index(locator2) + loc2
            xcoord.append(x[loc1:loc2-1])
            ycoord.append(x[loc2:loc3])
            #starLocations.append([float(xcoord),float(ycoord)])
#     tollerance = 5

#     xUpper = [float(x) + tollerance for x in xcoord]
#     xLower = [float(x) - tollerance for x in xcoord]
#     yUpper = [float(x) + tollerance for x in ycoord]
#     yLower = [float(x) - tollerance for x in ycoord]

    xcoordFloat = [float(x) for x in xcoord]
    ycoordFloat = [float(x) for x in ycoord]
    
    refDict = {'index':range(len(xcoord)),'X_IMAGE':xcoordFloat,'Y_IMAGE':ycoordFloat}
    refPandasNew = pd.DataFrame(data=refDict)
    table = Table.from_pandas(refPandasNew)
    return table

In [28]:
refTabTest = molePositions2Tab('/home/mj1e16/moleGazer/Photos/F/fitsImages/29018.reg')
alignTabTest = molePositions2Tab('/home/mj1e16/moleGazer/Photos/F/fitsImages/79948.reg')

In [29]:
alignTabTest['X_IMAGE'][0]

726.92286999999999

In [ ]:
makeDS9RegFile([alignTab],'/home/mj1e16/moleGazer/MoleGazer/79948Regions','sex')
makeDS9RegFile([refTab],'/home/mj1e16/moleGazer/MoleGazer/29018Regions','sex')

In [4]:
# could use a re-work

def findTestSources(sources,lowx, highx, lowy, highy, imageNo):
    """
    Searches for bright objects to be used for pattern recognition within a specified region of the reference image.

    :param sources: Table of object data
    :param lowx: Lower X limit of for searching region region
    :param highx: Upper X limit
    :param lowy: Lower Y limit
    :param highy: Upper Y limit
    :param imageNo: Reference image number
    :return: Indexes in sources of (up to) the brightest 15 objects contained within the specified region
    """
    testsources = {}
    mag = []
    number = []
    #for x in range(sources[imageNo][-1][0]):
    for x in range(len(sources[imageNo])):
        x1 = sources[imageNo]['X_IMAGE'][x]
        y1 = sources[imageNo]['Y_IMAGE'][x]
        if x1 > lowx and x1 < highx and y1 > lowy and y1 < highy:
            testsources[x] = sources[imageNo][x]
            mag.append(sources[imageNo]['MAG_BEST'][x])
            number.append(x)
    Z2 = [x for _,x in sorted(zip(mag,number))]
    samplestars = Z2[0:15]
    
    #astroprov.provcall([sources,lowx, highx, lowy, highy, imageNo],[samplestars],"findTestSources_PythonCode2PythonCode_SQ_tmpl.provn","findTestSources")
    return samplestars

In [5]:
def getImageData(imageName,extension):
    hdu = fits.open(imageName)
    imageData = hdu[extension].data
    return imageData

In [6]:
def ds9File2Lists(fileName):
    with open(fileName,'r') as f:
        data = f.readlines()

    ycoord = []
    xcoord = []
    radius = []
    starLocations = []
    for x in data[:-1]:
        try:
            locator1 = '('
            locator2 = ','
            locator3 = ')'
            loc1 = x.index(locator1) +1
            #print(x)
            loc2 = x[loc1:].index(locator2) + loc1 +1
            loc3 = x[loc2:].index(locator2) + loc2
            loc4 = x.index(locator3)
            xcoord.append(float(x[loc1:loc2-1]))
            ycoord.append(float(x[loc2:loc3]))
            radius.append(float(x[loc3+1:loc4]))
        except:
            print('Line - {}'.format(x))
    return [xcoord,ycoord,radius]

In [7]:
def plotMolesOnly(image,moleLocFile):
    imdata = getImageData(image,0)
    moleLocs = ds9File2Lists(moleLocFile)
    alignlocs = moleLocs
    mask = []
    #maskData = []
    for mole in range(len(alignlocs[0])):
        centre = PixCoord(alignlocs[0][mole],alignlocs[1][mole])
        reg = CirclePixelRegion(centre,alignlocs[2][mole])
        mask.append(reg.to_mask())
        #maskData.append(mask[mole].cutout(imdata)*mask[mole].data)
    
    fig, [ax1,ax2] = plt.subplots(1,2,figsize=(10,20))
    ax1.imshow(imdata)
    for x in range(len(mask)):
        numbering = mpl_text.Text(x=alignlocs[0][x],y=alignlocs[1][x],text=str(x),color='r')
        ax1.add_artist(mask[x].bbox.as_artist(facecolor='none', edgecolor='white'))
        ax1.add_artist(numbering)
    #ax2.imshow(zeros)
    ax2.imshow(imdata)
    plt.show()

In [8]:
def cosineRuleRounded(masterStar,star1,star2):
    a = ((masterStar[0]-star2[0])**2 + (masterStar[1]-star2[1])**2)**0.5
    b = ((star1[0]-masterStar[0])**2 + (star1[1]-masterStar[1])**2)**0.5
    c = (star1[0]-star2[0])**2 + (star1[1]-star2[1])**2
    C = np.arccos((a**2 + b**2 - c)/(2*a*b))
    CRounded = round(C,1)
    return CRounded

In [9]:
def cosineRule(masterStar,star1,star2):
    a = ((masterStar[0]-star2[0])**2 + (masterStar[1]-star2[1])**2)**0.5
    b = ((star1[0]-masterStar[0])**2 + (star1[1]-masterStar[1])**2)**0.5
    c = (star1[0]-star2[0])**2 + (star1[1]-star2[1])**2
    C = np.arccos((a**2 + b**2 - c)/(2*a*b))
    #CRounded = round(C,1)
    return C

In [10]:
def findAllAnglesPerStar(table,starNo):
    masterStar = [table['X_IMAGE'][starNo],table['Y_IMAGE'][starNo]]
    allAngles = []
    for x in range(len(table)):
        if x != starNo:
            anglePerStar = []
            star1 = [table['X_IMAGE'][x],table['Y_IMAGE'][x]]
            for y in range(len(table)):
                if y != x and y!= starNo:
                    star2 = [table['X_IMAGE'][y],table['Y_IMAGE'][y]]
                    anglePerStar.append(cosineRule(masterStar,star1,star2))
            allAngles.append(anglePerStar)
    return allAngles

In [11]:
def constructDataFrame(table,masterNo,imNo,angleDif=0.017,distanceDiff=10):
    masterStarPos = [table['X_IMAGE'][masterNo],table['Y_IMAGE'][masterNo]]
    xPositions = table['X_IMAGE'].tolist()
    yPositions = table['Y_IMAGE'].tolist()
    indexes = range(len(xPositions))
    # remove master star from list
    
    dfName = 'imNo{}mNo{}'.format(imNo,masterNo)
    
    angles = []
    distances = []
    distances2 = []
    distances3 = []
    id1 = []
    id2 = []
    for starNo in range(len(xPositions)):
        if starNo != masterNo:
            angles.extend([cosineRule(masterStarPos,[xPositions[starNo],yPositions[starNo]],[x2,y2]) for x2,y2,i in zip(xPositions,yPositions,indexes) if i!=starNo and i!=masterNo])
            distances.extend([distanceFromImagePosition(xPositions[starNo],yPositions[starNo],x2,y2) for x2,y2,i in zip(xPositions,yPositions,indexes) if i!=starNo and i!=masterNo])
            distances2.extend([distanceFromImagePosition(masterStarPos[0],masterStarPos[1],x2,y2) for x2,y2,i in zip(xPositions,yPositions,indexes) if i!=starNo and i!=masterNo])
            distances3.extend([distanceFromImagePosition(xPositions[starNo],yPositions[starNo],masterStarPos[0],masterStarPos[1]) for i in indexes if i!=starNo and i!=masterNo])
            id1.extend([starNo for x in range(len(xPositions)-2)])
            id2.extend([i for i in indexes if i!=starNo and i!=masterNo])
        
    anglesPlus = [x+angleDif for x in angles]
    anglesMinus = [x-angleDif for x in angles]
    
    distancesPlus = [x+distanceDiff for x in distances]
    distancesMinus = [x-distanceDiff for x in distances]
    
    distances2Plus = [x+distanceDiff for x in distances2]
    distances2Minus = [x-distanceDiff for x in distances2]
    
    distances3Plus = [x+distanceDiff for x in distances3]
    distances3Minus = [x-distanceDiff for x in distances3]
    
    bigList = [id1,id2,angles,anglesPlus,anglesMinus,distances,distancesPlus,distancesMinus,distances2,distances2Plus,distances2Minus,distances3,distances3Plus,distances3Minus]
    bigListLength = [len(x) for x in bigList]
    #print(bigListLength)
    
    finalDF = pd.DataFrame(np.column_stack([id1,id2,angles,anglesPlus,anglesMinus,distances,distancesPlus,distancesMinus,
                                            distances2,distances2Plus,distances2Minus,distances3,distances3Plus,distances3Minus]),
                          columns=['ID1imNo{}mNo{}'.format(imNo,masterNo),'ID2imNo{}mNo{}'.format(imNo,masterNo),
                                   'AnglesimNo{}mNo{}'.format(imNo,masterNo),'AnglesPlusimNo{}mNo{}'.format(imNo,masterNo),
                                   'AnglesMinusimNo{}mNo{}'.format(imNo,masterNo),'Distance1imNo{}mNo{}'.format(imNo,masterNo),
                                   'Distance1PlusimNo{}mNo{}'.format(imNo,masterNo),'Distance1MinusimNo{}mNo{}'.format(imNo,masterNo),
                                   'Distance2imNo{}mNo{}'.format(imNo,masterNo),'Distance2PlusimNo{}mNo{}'.format(imNo,masterNo),
                                   'Distance2MinusimNo{}mNo{}'.format(imNo,masterNo),'Distance3imNo{}mNo{}'.format(imNo,masterNo),
                                   'Distance3PlusimNo{}mNo{}'.format(imNo,masterNo),'Distance3MinusimNo{}mNo{}'.format(imNo,masterNo)])
    return [finalDF,dfName]

In [12]:
def generateSQLcode(df1,df2,dfName1,dfName2):
    sqlcode = '''
    SELECT *
    FROM {0}
    JOIN {1}
    ON {0}.Angles{2} < {1}.AnglesPlus{3}
    AND {0}.Angles{2} > {1}.AnglesMinus{3}
    AND {0}.Distance1{2} < {1}.Distance1Plus{3}
    AND {0}.Distance1{2} > {1}.Distance1Minus{3}
    AND {0}.Distance2{2} < {1}.Distance2Plus{3}
    AND {0}.Distance2{2} > {1}.Distance2Minus{3}
    AND {0}.Distance3{2} < {1}.Distance3Plus{3}
    AND {0}.Distance3{2} > {1}.Distance3Minus{3}
    '''.format(df1,df2,dfName1,dfName2)
    return sqlcode

In [78]:
sqlcode = '''
SELECT *
FROM {0}
JOIN {1}
ON {0}.Angles{2} < {1}.AnglesPlus{3}
AND {0}.Angles{2} > {1}.AnglesMinus{3}
AND {0}.Distance1{2} < {1}.Distance1Plus{3}
AND {0}.Distance1{2} > {1}.Distance1Minus{3}
AND {0}.Distance2{2} < {1}.Distance2Plus{3}
AND {0}.Distance2{2} > {1}.Distance2Minus{3}
AND {0}.Distance3{2} < {1}.Distance3Plus{3}
AND {0}.Distance3{2} > {1}.Distance3Minus{3}
'''.format('testDF','miniDF',dfName,dfName)

In [45]:
masterNo=0
refDF = constructDataFrame(refTabTest,masterNo,0)
dfName =  refDF[1]
refDF = refDF[0]
dfLengths = []

t0 = time.time()

for x in range(len(alignTabTest)):
    tempDF = constructDataFrame(alignTabTest,x,1)
    dfName2 = tempDF[1]
    tempDF = tempDF[0]
    sqlcode =  generateSQLcode('refDF','tempDF',dfName,dfName2)
    newdf = ps.sqldf(sqlcode,locals())
    dfLengths.append(len(newdf))
    print(x)

print('Time = {}'.format(time.time()-t0))
bestMatches = [x for x,i in enumerate(dfLengths) if i == max(dfLengths)]
print(bestMatches)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
Time = 1815.02982116
[0]


In [49]:
tempDF = constructDataFrame(alignTabTest,bestMatches[0],1)
dfName2 = tempDF[1]
tempDF = tempDF[0]
sqlcode =  generateSQLcode('refDF','tempDF',dfName,dfName2)
newdf = ps.sqldf(sqlcode,locals())
dfLengths.append(len(newdf))

In [54]:
newdf
idDF = newdf[['ID1imNo0mNo0','ID2imNo0mNo0','ID1imNo1mNo0','ID2imNo1mNo0']]

In [55]:
idDF

,ID1imNo0mNo0,ID2imNo0mNo0,ID1imNo1mNo0,ID2imNo1mNo0
0,2.0,11.0,2.0,12.0
1,2.0,12.0,2.0,13.0
2,2.0,26.0,2.0,22.0
3,2.0,31.0,2.0,21.0
4,2.0,45.0,2.0,20.0
5,2.0,50.0,2.0,29.0
6,2.0,55.0,2.0,28.0
7,2.0,72.0,2.0,87.0
8,2.0,81.0,2.0,69.0
9,2.0,94.0,2.0,27.0


In [40]:
df = refTabTest.to_pandas()
mole = df[(df['X_IMAGE']>350) & (df['X_IMAGE']<380) & (df['Y_IMAGE']>3400) & (df['Y_IMAGE']<3500)]
mole

,X_IMAGE,Y_IMAGE,index
0,367.61031,3428.1942,0


In [56]:
refTabTest[2]

X_IMAGE,Y_IMAGE,index
float64,float64,int64
628.57482,3428.8189,2


In [57]:
alignTabTest[2]

X_IMAGE,Y_IMAGE,index
float64,float64,int64
987.85087,3427.2612,2


In [90]:
bestMatches = [x for x,i in enumerate(dfLengths) if i == max(dfLengths)]
print(bestMatches)

[516]


In [32]:
print(max(dfLengths))

228


In [16]:
alignTab[43]

NUMBER,MAG_BEST,FLUX_MAX,X_IMAGE,Y_IMAGE,ELONGATION,ELLIPTICITY
,mag,ct,pix,pix,,
int64,float64,float64,float64,float64,float64,float64
44,-10.3691,42.02203,655.0755,413.3113,4.675,0.786


In [103]:
refTab[44]

NUMBER,MAG_BEST,FLUX_MAX,X_IMAGE,Y_IMAGE,ELONGATION,ELLIPTICITY
,mag,ct,pix,pix,,
int64,float64,float64,float64,float64,float64,float64
45,-11.4605,72.84969,366.3849,3430.3481,1.317,0.24


In [76]:
testDF = constructDataFrame(refTab,0,0)
dfName = testDF[1]
testDF = testDF[0]

[2652, 2652, 2652, 2652, 2652, 2652, 2652, 2652, 2652, 2652, 2652, 2652, 2652, 2652]


In [77]:
miniDF = testDF[0:10]

In [79]:
newdf = ps.sqldf(sqlcode,locals())

In [80]:
newdf

,ID1imNo0mNo0,ID2imNo0mNo0,AnglesimNo0mNo0,AnglesPlusimNo0mNo0,AnglesMinusimNo0mNo0,Distance1imNo0mNo0,Distance1PlusimNo0mNo0,Distance1MinusimNo0mNo0,Distance2imNo0mNo0,Distance2PlusimNo0mNo0,...,AnglesMinusimNo0mNo0,Distance1imNo0mNo0,Distance1PlusimNo0mNo0,Distance1MinusimNo0mNo0,Distance2imNo0mNo0,Distance2PlusimNo0mNo0,Distance2MinusimNo0mNo0,Distance3imNo0mNo0,Distance3PlusimNo0mNo0,Distance3MinusimNo0mNo0
0,1.0,2.0,0.759933,0.809933,0.709933,590.100945,600.100945,580.100945,854.695985,864.695985,...,0.709933,590.100945,600.100945,580.100945,854.695985,864.695985,844.695985,580.048466,590.048466,570.048466
1,1.0,3.0,1.042141,1.092141,0.992141,551.598890,561.598890,541.598890,523.638570,533.638570,...,0.992141,551.598890,561.598890,541.598890,523.638570,533.638570,513.638570,580.048466,590.048466,570.048466
2,1.0,4.0,0.052518,0.102518,0.002518,461.392515,471.392515,451.392515,118.862022,128.862022,...,0.002518,461.392515,471.392515,451.392515,118.862022,128.862022,108.862022,580.048466,590.048466,570.048466
3,1.0,5.0,1.853956,1.903956,1.803956,861.793806,871.793806,851.793806,495.584045,505.584045,...,1.803956,861.793806,871.793806,851.793806,495.584045,505.584045,485.584045,580.048466,590.048466,570.048466
4,1.0,6.0,0.268958,0.318958,0.218958,296.891346,306.891346,286.891346,305.448764,315.448764,...,0.218958,296.891346,306.891346,286.891346,305.448764,315.448764,295.448764,580.048466,590.048466,570.048466
5,1.0,7.0,0.923291,0.973291,0.873291,463.731633,473.731633,453.731633,381.665511,391.665511,...,0.873291,463.731633,473.731633,453.731633,381.665511,391.665511,371.665511,580.048466,590.048466,570.048466
6,1.0,8.0,2.313056,2.363056,2.263056,1241.540712,1251.540712,1231.540712,773.545718,783.545718,...,2.263056,1241.540712,1251.540712,1231.540712,773.545718,783.545718,763.545718,580.048466,590.048466,570.048466
7,1.0,9.0,3.127615,3.177615,3.077615,1331.616321,1341.616321,1321.616321,751.599838,761.599838,...,3.077615,1331.616321,1341.616321,1321.616321,751.599838,761.599838,741.599838,580.048466,590.048466,570.048466
8,1.0,10.0,2.849888,2.899888,2.799888,804.969153,814.969153,794.969153,231.950798,241.950798,...,2.799888,804.969153,814.969153,794.969153,231.950798,241.950798,221.950798,580.048466,590.048466,570.048466
9,1.0,11.0,2.477999,2.527999,2.427999,1567.107823,1577.107823,1557.107823,1068.883623,1078.883623,...,2.427999,1567.107823,1577.107823,1557.107823,1068.883623,1078.883623,1058.883623,580.048466,590.048466,570.048466


In [ ]:
[cosineRule(masterStarPos,[xPositions[starNo],yPositions[starNo]],[x2,y2]) for x2,y2,i in zip(xPositions,yPositions,indexes) if i!=starNo]

In [43]:
cosineRule([refTab[0]['X_IMAGE'],refTab[0]['Y_IMAGE']],[refTab[1]['X_IMAGE'],refTab[1]['Y_IMAGE']],[refTab[2]['X_IMAGE'],refTab[2]['Y_IMAGE']])

0.75993333310209743

In [14]:
def distanceFromImagePosition(x1,y1,x2,y2):
    D = (((abs(x1-x2)**2) + (abs(y1-y2)**2))**0.5)
    return D

In [10]:
def distance(sources,object1,object2):
    """
    Find the distance between two objects in an image.

    :param sources: Table of object data
    :param object1: Index of first object in sources
    :param object2: Index of second object in sources
    :param imageNo: Image Number
    :return: Diagonal distance between the two objects
    """
    D = (((abs(sources['X_IMAGE'][object1]-sources['X_IMAGE'][object2])**2) + (abs(sources['Y_IMAGE'][object1]-sources['Y_IMAGE'][object2])**2))**0.5)
    Drounded = round(D,-1)
    #print([round(x/5,-1)*5 for x in a])
    return Drounded


In [11]:
def referenceDistances(sources,samplestars,imageNo):
    """
    Calculate the diagonal distance between every object in samplestars.

    :param sources: Table of object data
    :param samplestars: List containing the indexs of stars
    :param imageNo: reference image number
    :return: List containing a list for each object, comprised of the diagonal distance between that object and every object in samplestars
    """
    #referencedistances = [[]]*len(samplestars)
    referencedistances = [[] for x in samplestars]
    for y in range(len(samplestars)):
        refd = []
        for x in range(len(samplestars)):
            refd.append(distance(sources,samplestars[y],samplestars[x],imageNo))
        referencedistances[y] = refd #.append(distance(sources,samplestars[y],samplestars[x],imageNo))
    
    #astroprov.provcall([sources,samplestars,imageNo],[referencedistances],"referenceDistances_PythonCode2PythonCode_SQ_tmpl.provn","referenceDistances")
    return referencedistances


In [12]:
def findFullDistancesPerObject(sources,objectNo):
    """
    Find the diagonal distances between every object in the image.

    :param sources: Table of object data
    :param imageno: Image number
    :return: Digonal distance between every object in the image
    """

    fulldistances = []
    for y in range(len(sources)):
        fulldistances.append(distance(sources,objectNo,y)) 
    
    #astroprov.provcall([sources,imageno],[fulldistances],"findFullDistances_PythonCode2PythonCode_SQ_tmpl.provn","findFullDistances")
    #fulldistances = [round(number/10,1) * 10 for number in fulldistances] # rounding can be changed by changing the mul/div
    return fulldistances

In [13]:
def matchAngles(allAnglesPerObject,allAngles,matchFraction=0.6):
    percentageForMatch = 0.9

    # keep for now, inner join on an sql table might be better... ask Age about this?

    translationDictionary = {}

    for i,obj in enumerate(allAnglesPerObject): # loop over each object's angle set
        oneObjectDict = {}
        oneObjectLengths = {}
        for j,angleSet in enumerate(obj): # loop over one angle set
            matchLengths = []
            for k,ogObj in enumerate(allAngles): # loop over master angle set
                #result = all(elem in angleSet for elem in ogObj) # here is the problem-need to relax these constraints and change to any 
                result = any(elem in angleSet for elem in ogObj) # here is the problem-need to relax these constraints and change to any 
                result = [x for x in angleSet if x in ogObj]
                #originalLength = len(list(set(ogObj)))
                newLength = len(list(set(result)))
                #if newLength > matchFraction*originalLength: # this step may be unecessary
                if j not in oneObjectDict.keys():    
                    oneObjectDict[j] = k
                    oneObjectLengths[j] = newLength
                else:
                    if newLength > oneObjectLengths[j]:
                        oneObjectDict[j] = k
                        
        translationDictionary[i] = oneObjectDict
    return translationDictionary # values are -1 of their true values - this is accounted for later

In [14]:
def matchAngles(allAnglesPerObject,allAngles):
    percentageForMatch = 0.9

    # keep for now, inner join on an sql table might be better... ask Age about this?

    translationDictionary = {}

    for i,obj in enumerate(allAnglesPerObject): # loop over each object's angle set
        oneObjectDict = {}
        for j,angleSet in enumerate(obj): # loop over one angle set
            matchLengths = []
            for k,ogObj in enumerate(allAngles): # loop over master angle set
                result = all(elem in angleSet for elem in ogObj) # here is the problem-need to relax these constraints and change to any 
                #result = any(elem in angleSet for elem in ogObj) # here is the problem-need to relax these constraints and change to any 
                
                if result:
                    oneObjectDict[j] = k
        translationDictionary[i] = oneObjectDict
    return translationDictionary # values are -1 of their true values - this is accounted for later

In [15]:
# def confirmAngleMatchWithDistances(translationDictionary,refTab,referenceDistances,indexing):
#     trueMatchDict = {}
#     potentialMatches = []
#     potentialRef = []
#     for key,value in translationDictionary.items():
#         oneMatch = []
#         oneRef = []
#         if len(value) !=0:
#             for imgNo,sampleNo in value.items():
#                 objectDistance = distance(refTab,key,imgNo+1)
#                 if objectDistance == referenceDistances[indexing.index(sampleNo)]:
#                     oneMatch.append(imgNo+1)
#                     oneRef.append(sampleNo)
#         potentialMatches.append(oneMatch)
#         potentialRef.append(oneRef)

#     listLengths = [len(x) for x in potentialMatches]
#     bestMatch = max(listLengths)
#     bestMatch = [i for i,j in enumerate(listLengths) if j==bestMatch]
#     if len(bestMatch) > 1:
#         print('Currently {} potentail candidates, choosing the first'.format(len(bestMatch)))
#     trueMatchDict[0] = bestMatch[0]
#     for x in range(len(potentialMatches[bestMatch[0]])):
#         trueMatchDict[potentialRef[bestMatch[0]][x]] = potentialMatches[bestMatch[0]][x]
#     return trueMatchDict

In [16]:
def confirmAngleMatchWithDistances(translationDictionary,refTab,referenceDistances,indexing,closeness=20):
    trueMatchDict = {}
    potentialMatches = []
    potentialRef = []
    for key,value in translationDictionary.items():
        oneMatch = []
        oneRef = []
        if len(value) !=0:
            for imgNo,sampleNo in value.items():
                objectDistance = distance(refTab,key,imgNo)
                refDist = referenceDistances[indexing.index(sampleNo)]
                if objectDistance > (refDist-closeness) and objectDistance < (refDist+closeness):
                    oneMatch.append(imgNo)
                    oneRef.append(sampleNo)
        potentialMatches.append(oneMatch)
        potentialRef.append(oneRef)

    listLengths = [len(x) for x in potentialMatches]
    bestMatch = max(listLengths)
    bestMatch = [i for i,j in enumerate(listLengths) if j==bestMatch]
    if len(bestMatch) > 1:
        print('Currently {} potentail candidates, choosing the first'.format(len(bestMatch)))
    trueMatchDict[0] = bestMatch[0]
    for x in range(len(potentialMatches[bestMatch[0]])):
        trueMatchDict[potentialRef[bestMatch[0]][x]] = potentialMatches[bestMatch[0]][x]
    return trueMatchDict

In [16]:
#testSources = findTestSources([refTab],0,10000,0,10000,0)

In [17]:
# targetMoles = [44,37,35,34,40,41,20,16,15,14,10]
# targetMolesReal = [x+1 for x in targetMoles] 
# fullListofObjects = range(len(refTab))
# toBeRemoved = [x for x in fullListofObjects if x not in targetMoles]
# #print(len(fullListofObjects),len(toBeRemoved),len(testSources))
# refTab.remove_rows(toBeRemoved)
# sampleStars = refTab
# refTab = Table.read('/home/mj1e16/moleGazer/Photos/F/fitsImages/simMolePos/29018InvertedAverageCropped166.44444444444_21.8421052632_gauss_5.0_9x9.conv.cat',format='ascii.sextractor')

In [21]:
allAngles = findAllAnglesPerStar(refTab,0) # all angles between master and sample stars
#allAngles = findAllAnglesPerStar(sampleStars,0) # all angles between master and sample stars

allAnglesPerObject = [] # nested list - each list represents one star chosen as the master star and the values within it are the angles between each other star
for x in range(len(refTab)):
    allAnglesPerObject.append(findAllAnglesPerStar(refTab,x))


In [24]:
dictionary = {1:'a',2:'b'}
print(dictionary.keys())

[1, 2]


In [22]:
translationDict = matchAngles(allAnglesPerObject,allAngles)

In [23]:
# referenceDistances = []
# indexing = []
# for x in range(1,len(sampleStars)):
#     referenceDistances.append(distance(sampleStars,0,x))
#     indexing.append(x-1)

referenceDistances = []
indexing = []
for x in range(len(refTab)):
    referenceDistances.append(distance(refTab,0,x))
    indexing.append(x)

In [24]:
newtrueMatchDict = confirmAngleMatchWithDistances(translationDict,refTab,referenceDistances,indexing,closeness=50)

In [25]:
newtrueMatchDict

{0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 5,
 6: 6,
 7: 7,
 10: 10,
 11: 11,
 15: 15,
 17: 17,
 18: 18,
 20: 20,
 30: 30,
 32: 32,
 33: 33,
 36: 36,
 38: 38,
 41: 41,
 42: 42,
 43: 43,
 45: 45,
 46: 46,
 47: 47,
 48: 48,
 49: 49,
 50: 50,
 51: 51}

In [26]:
translationDict[0]

{0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 5,
 6: 6,
 7: 7,
 8: 51,
 9: 43,
 10: 10,
 11: 11,
 12: 49,
 13: 41,
 14: 15,
 15: 15,
 16: 50,
 17: 17,
 18: 18,
 19: 48,
 20: 20,
 21: 45,
 22: 48,
 23: 50,
 24: 51,
 25: 51,
 26: 48,
 27: 32,
 28: 49,
 29: 48,
 30: 30,
 31: 43,
 32: 32,
 33: 33,
 34: 41,
 35: 48,
 36: 36,
 37: 46,
 38: 38,
 39: 50,
 40: 50,
 41: 41,
 42: 42,
 43: 43,
 44: 48,
 45: 45,
 46: 46,
 47: 47,
 48: 48,
 49: 49,
 50: 50,
 51: 51}